## Import Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

## Set Paths & Configs

In [ ]:
excel_path = '/kaggle/input/huron-dataset/image_class_mapping.xlsx'
masks_dir = '/kaggle/input/huron-dataset/Sliced_masks' 
mapped_df = pd.read_excel(excel_path)

#verify dataset count
print(f"Total mapped images: {len(mapped_df)}")
print(mapped_df.head())

#GPU configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Prepocessing Data

### Splitting Dataset

In [ ]:
all_mask_files = os.listdir(masks_dir)

#separate mapped and unmapped masks
unmapped_files = [f for f in all_mask_files if f not in mapped_df['Image'].values]

#split mapped dataset
train_mapped, test_mapped = train_test_split(
    mapped_df, test_size=0.2, random_state=42, stratify=mapped_df['Class'])

#split sizes
print(f"Training Mapped: {len(train_mapped)}, Test Mapped: {len(test_mapped)}")
print(f"Unmapped Files: {len(unmapped_files)}")


### Dataset Class

In [ ]:
#We create a datset class to help load and preprocess data

class MaskDataset(Dataset):
    def __init__(self, dataframe, masks_dir, transform=None):
        self.dataframe = dataframe
        self.masks_dir = masks_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        mask_path = os.path.join(self.masks_dir, row['Image'])
        mask = np.array(Image.open(mask_path).convert("L")) #grayscale
        label = row['Class']

        if self.transform:
            mask = self.transform(mask)

        return mask, label

#transform the masks
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485], std=[0.229]),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))])

#split the datasets
train_dataset = MaskDataset(train_mapped, masks_dir, transform)
test_dataset = MaskDataset(test_mapped, masks_dir, transform)

## Model Setup & Training

### Setup

In [ ]:
#loading the model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(mapped_df['Class'].unique())) #gets the length of each unique class which is 7
model = model.to(device)

#loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#model summary
print(model)


### Training

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

unique_classes = set()
for _, labels in train_loader:
    unique_classes.update(labels)
    
class_to_int = {cls: idx for idx, cls in enumerate(unique_classes)}
int_to_class = {idx: cls for cls, idx in class_to_int.items()}

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=None):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            labels = [class_to_int[label] for label in labels]
            inputs, labels = inputs.to(device), torch.tensor(labels).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}")

        #call test
        test_model(model, test_loader, criterion)

def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            labels = [class_to_int[label] for label in labels]
            inputs, labels = inputs.to(device), torch.tensor(labels).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Loss: {test_loss / len(test_loader)}, Accuracy: {100 * correct / total}%")

#Train the model
train_model(model, train_loader, test_loader, criterion, optimizer, epochs=15)

## Predict Classes

In [ ]:
def predict_unmapped_masks(model, unmapped_files, masks_dir, transform, int_to_class, output_path="mapped_results.xlsx"):
    model.eval()
    predictions = []
    confidence_scores = []

    for file in tqdm(unmapped_files):
        mask_path = os.path.join(masks_dir, file)
        mask = np.array(Image.open(mask_path).convert("L"))
        if transform:
            mask = transform(mask).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(mask)
            probs = torch.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probs, 1)
            predictions.append(predicted.item())
            confidence_scores.append(confidence.item())

    predicted_classes = [int_to_class[p] for p in predictions]

    #confidence levels
    confidence_categories = ['High' if c >= 0.6 else 'Medium' if c >= 0.3 else 'Low' for c in confidence_scores]

    #output structure
    output_df = pd.DataFrame({
        'Image': unmapped_files,
        'Predicted Class': predicted_classes,
        'Confidence Score': confidence_scores,
        'Confidence Category': confidence_categories
    })

    output_df.to_excel(output_path, index=False)
    print(f"Output saved to {output_path}")

#find predictions
predict_unmapped_masks(model, unmapped_files, masks_dir, transform,int_to_class)
